In [86]:
import pandas as pd
import numpy as np
import pymysql as ps
import re
import xlrd


db_host = 'localhost'
db_user = 'root'
db_pwd = '521521'
db_name = 'sxzy'
char_type = 'utf-8'
db = ps.connect(db_host, db_user, db_pwd, db_name, charset='utf8')
couser = db.cursor()

In [87]:
# 获取交易卡号信息
sql='select distinct  cxkh,cxzh,jymc,jydfzkh,jydfmc from gas_bank_records'
couser.execute(sql)
rows = couser.fetchall()

In [88]:
df_account=pd.DataFrame(rows,columns=['cxkh','cxzh','jymc','jydfzkh','jydfmc'])

In [89]:
# 交易方账号信息
df_jy_account=df_account[['jymc','cxkh']].drop_duplicates()
df_jy_account.head()

,jymc,cxkh
0,杨建进,6222080509000555448
1,刘兵,6222020509007272982
5,闫二伟,509256801204024445
14,高明明,6212880509000119818
29,刘爱萍,6222080509000160017


In [90]:
#交易对方账号信息
df_jydf_account=df_account[['jydfmc','jydfzkh']].drop_duplicates()
df_jydf_account.head()
df_jydf_account.columns=['jymc','cxkh']

In [91]:
#总的交易账号信息 已调单+未调单
df_account_fin=pd.concat([df_jy_account,df_jydf_account],axis=0).drop_duplicates()
df_account_fin.head()

,jymc,cxkh
0,杨建进,6222080509000555448
1,刘兵,6222020509007272982
5,闫二伟,509256801204024445
14,高明明,6212880509000119818
29,刘爱萍,6222080509000160017


In [92]:
#获取嫌疑人姓名
sql_relactives='select jymc from relatives'
couser.execute(sql_relactives)
rows = couser.fetchall()
relatives_list=[i[0] for i in rows]


In [93]:
len(relatives_list)

78

In [94]:
#获取嫌疑人的所有账号信息
df_diaodan=df_account_fin[df_account_fin['jymc'].isin(relatives_list)]
cxkh_list=np.array(df_jy_account['cxkh'])
# 判断嫌疑人的账号是否调单
df_diaodan['y_n_diaodan']=df_diaodan['cxkh'].apply(lambda x : '已调单' if x in cxkh_list else '未调单')

In [95]:
# df_diaodan.columns=['交易名称','交易卡号','是否调单']
# df_diaodan.to_excel('c:/Users/jxh/Desktop/嫌疑人及其亲属调单情况.xlsx',index=False,encoding='utf-8')

In [97]:
df_diaodan

,jymc,cxkh,y_n_diaodan
0,杨建进,6222080509000555448,已调单
5,闫二伟,509256801204024445,已调单
29,刘爱萍,6222080509000160017,已调单
40,刘爱萍,509256801202254682,已调单
95,崔新平,509256801202467393,已调单
...,...,...,...
138900,吕梁市建筑安装总公司第四工程处,553281010300000004015,未调单
138902,吕梁市建筑安装总公司第四工程处,692138560,未调单
138904,吕梁市建筑安装总公司第四工程处,543000001018010684167,未调单
138962,吕梁市建筑安装总公司第四工程处,144225733185,未调单


In [98]:
# 非嫌疑人情况
fxyr_list=['高明明','崔爱明','吴晓虎','孟庆冬','乔和平','乔秀秀','陈月娥']

In [101]:
# 判断所有账单中的非嫌疑人的账号明细
df_fei_diaodan=df_account_fin[df_account_fin['jymc'].isin(fxyr_list)]
cxkh_list=np.array(df_jy_account['cxkh'])
# 判断所有账单中的非嫌疑人的调单明细
df_fei_diaodan['y_n_diaodan']=df_fei_diaodan['cxkh'].apply(lambda x : '已调单' if x in cxkh_list else '未调单')
df_fei_diaodan.columns=['交易名称','交易卡号','是否调单']

D:\develop\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [103]:
df_fei_diaodan.to_excel('c:/Users/jxh/Desktop/非嫌疑人调单情况.xlsx',index=False,encoding='utf-8')